In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importing Data 
raw_train_data = pd.read_csv('data/train_data.csv')
raw_test_data = pd.read_csv('data/test.csv')
raw_train_data.head()

,ofd_date,country_code,fc_codes,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline
0,2021-06-30,C,"F6, F8, F14, F17",D33,14594,14568,782,896,615,767,35,2,4
1,2021-06-30,C,"F6, F8, F9, F14, F17, F18",D37,12736,13111,655,823,211,29,17,2,1
2,2021-06-30,C,"F1, F4, F6, F7, F13, F15, F16",D34,14562,15651,1028,1910,225,35,47,3,1
3,2021-06-30,C,"F2, F6, F7, F10, F12, F13, F14, F15, F19",D45,11165,11467,514,769,56,39,29,0,1
4,2021-06-30,C,"F6, F8, F13, F14, F17",D50,10006,10423,399,842,52,60,65,1,1


In [3]:
# Simple descriptive analysis 
raw_train_data[['Earlies_Exp','MNR_SNR_Exp']].describe()

,Earlies_Exp,MNR_SNR_Exp
count,11309.000000,11309.000000
mean,644.620656,215.780175
std,604.513009,475.355339
min,0.000000,1.000000
25%,220.000000,76.000000
50%,463.000000,120.000000
75%,867.000000,208.000000
max,4934.000000,17411.000000


In [4]:
def set_data_types(data):
    '''Changes data types to correct type.'''
    
    data['ofd_date'] = pd.to_datetime(data.ofd_date, format='%Y-%m-%d')
    return data

def create_target(data):
    '''Creates target variable (Earlies_Exp - MNR_SNR_Exp) and drops those variables.'''
    
    data['target'] = data['Earlies_Exp'] - data['MNR_SNR_Exp']
    data = data.drop(['Earlies_Exp','MNR_SNR_Exp'], axis=1)
    return data


def apply_preprocessing(data, train=True):
    '''Takes all basic preprocessing functions and applies them in one function. Set train to false for test set as no target can be created.'''
    
    data = set_data_types(data)
    if train:
        data = create_target(data)
    return data[::-1]

train_data = apply_preprocessing(raw_train_data)
test_data = apply_preprocessing(raw_test_data, train=False)

In [15]:
train_data.iloc[train_data['target'].argmin()]

ofd_date            2021-02-10 00:00:00
country_code                          B
fc_codes        F24, F25, F26, F28, F29
station_code                        D12
OFD                               13341
Slam                              33195
Earlies_Rec                        2469
Rollover                            578
Returns                             137
R_Sideline                            0
Sideline                              2
target                           -16625
Name: 10567, dtype: object

In [5]:
# Descriptive stats about the Target variable
train_data['target'].describe()

count    11309.000000
mean       428.840481
std        718.831072
min     -16625.000000
25%         88.000000
50%        298.000000
75%        660.000000
max       4878.000000
Name: target, dtype: float64

In [6]:
# A closer look at the cases where the target goes below 0 
target_less_than_0 = train_data[train_data['target'] < 0]
print(f'There are {target_less_than_0.shape[0]} records where the target is less than 0')

There are 1257 records where the target is less than 0


In [7]:
target_less_than_0.describe()

,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,target
count,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000
mean,13815.030231,14208.854415,470.323787,403.350040,174.700080,281.784407,139.557677,-423.171042
std,7125.722862,7171.841754,532.217879,1169.987883,378.882534,613.391494,409.032198,1135.161292
min,0.000000,134.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16625.000000
25%,9135.000000,9348.000000,119.000000,45.000000,36.000000,0.000000,0.000000,-321.000000
50%,13109.000000,13456.000000,270.000000,86.000000,75.000000,4.000000,1.000000,-82.000000
75%,16708.000000,17029.000000,635.000000,229.000000,135.000000,318.000000,74.000000,-28.000000
max,44752.000000,45278.000000,3937.000000,17730.000000,4444.000000,7211.000000,3426.000000,-1.000000


In [9]:
# Experiment 1 - Treating all data points that exceed 4 standard deviations from the mean as outliers

upper_limit = train_data['target'].mean() + 4*train_data['target'].std()
lower_limit = train_data['target'].mean() - 4*train_data['target'].std()
train_with_limits = train_data[(train_data['target'] <= upper_limit) & (train_data['target'] >= lower_limit)]
train_outside_limits = train_data[(train_data['target'] > upper_limit) | (train_data['target'] < lower_limit)]
print(f'There are {train_data.shape[0] - train_with_limits.shape[0]} records that fall outside 4 standard deviations')

There are 84 records that fall outside 4 standard deviations


In [10]:
# Total number of data points for every country
train_data['country_code'].value_counts()

D    3000
B    2954
A    2929
C    2426
Name: country_code, dtype: int64

In [11]:
# Total number of data points for every station code
train_data['station_code'].value_counts()

D57    150
D52    150
D59    150
D8     150
D6     150
      ... 
D40    121
D47    121
D41    121
D46    121
D33    121
Name: station_code, Length: 80, dtype: int64

In [12]:
# Taking a closer look at the countries and stations that fall outside the boundary set above
train_outside_limits['country_code'].value_counts()
train_outside_limits['station_code'].value_counts()

D15    11
D11     6
D18     5
D14     5
D12     5
D13     5
D17     4
D47     4
D31     3
D41     2
D75     2
D1      2
D23     2
D22     2
D50     2
D38     2
D77     1
D71     1
D19     1
D29     1
D16     1
D73     1
D34     1
D26     1
D33     1
D37     1
D45     1
D32     1
D49     1
D48     1
D43     1
D39     1
D36     1
D35     1
D40     1
D46     1
D44     1
D27     1
Name: station_code, dtype: int64

- The station code with the most number of data points that fall outside 4 std deviations is D15 with 11
- The country code with the most number of data points that fall outside 4 std deviations is B with 50

In [13]:
train_with_limits['target'].describe()

count    11225.000000
mean       436.349042
std        568.302025
min      -2434.000000
25%         89.000000
50%        298.000000
75%        656.000000
max       3282.000000
Name: target, dtype: float64

In [55]:
train_within_limits

,ofd_date,country_code,fc_codes,station_code,OFD,Slam,Earlies_Rec,Rollover,Returns,R_Sideline,Sideline,target
11308,2021-02-01,D,"F35, F40, F42, F43, F46, F48, F52, F53, F55, F56",D57,18645,18982,1177,145,126,356,2,249
11307,2021-02-01,D,"F35, F36, F40, F42, F43, F46, F48, F49, F52, F...",D59,16828,16504,333,135,112,256,1,165
11306,2021-02-01,D,"F35, F48, F52, F53, F54, F56",D69,12479,12345,196,64,62,136,0,99
11305,2021-02-01,D,"F35, F36, F37, F38, F39, F40, F42, F43, F44, F...",D54,18986,18861,446,100,107,248,1,235
11304,2021-02-01,D,"F35, F37, F38, F39, F40, F42, F43, F44, F46, F...",D60,16025,15768,383,69,195,15,1,425
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-06-30,C,"F6, F8, F13, F14, F17",D50,10006,10423,842,60,65,1,1,347
3,2021-06-30,C,"F2, F6, F7, F10, F12, F13, F14, F15, F19",D45,11165,11467,769,39,29,0,1,458
2,2021-06-30,C,"F1, F4, F6, F7, F13, F15, F16",D34,14562,15651,1910,35,47,3,1,803
1,2021-06-30,C,"F6, F8, F9, F14, F17, F18",D37,12736,13111,823,29,17,2,1,444


In [56]:
# Creating a new csv file with the outliers removed
train_within_limits.to_csv('data/train_after_outlier.csv', index=False)

# Submission File

In [31]:
submission = pd.read_csv('data/SampleSubmission.csv')
submission.head()

,Id,Expected
0,2021-07-31_D33,200
1,2021-07-31_D37,281
2,2021-07-31_D34,312
3,2021-07-31_D45,44
4,2021-07-31_D50,-20


In [37]:
submission[submission['Expected'] < -2434]

,Id,Expected
988,2021-07-19_D75,-3024
